In [5]:
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

In [6]:
def get_acum(pm1, pm2, Temp_df):
    
    Temp_df['PM1'] = Temp_df['Average'].rolling(pm1).mean()
    Temp_df['PM2'] = Temp_df['Average'].rolling(pm2).mean()
    Temp_df.dropna(inplace = True)
    Temp_df['Posicion'] = np.where(Temp_df['PM1'] > Temp_df['PM2'], 1, -1)
    Temp_df['Retornos'] = np.log(Temp_df['Average']/Temp_df['Average'].shift(1))
    Temp_df.dropna(inplace = True)
    Temp_df['Estrategia'] = Temp_df['Retornos']*Temp_df['Posicion'].shift(1)
    Temp_df.dropna(inplace = True)
    # Temp_df['Retacum'] = Temp_df['Retornos'].cumsum().apply(np.exp)
    Temp_df['Estracum'] = Temp_df['Estrategia'].cumsum().apply(np.exp)
    
    return Temp_df['Estracum'].tail(1).copy()

In [7]:
def best_strategy(pm1, pm2, Bitso):
    
    Bitso['PM1'] = Bitso['Average'].rolling(pm1).mean()
    Bitso['PM2'] = Bitso['Average'].rolling(pm2).mean()
    Bitso.dropna(inplace = True)
    Bitso.plot(figsize=(12,8))
    Bitso['Posicion'] = np.where(Bitso['PM1'] > Bitso['PM2'], 1, -1)
    Bitso.plot(figsize=(12,8), secondary_y = 'Posicion')
    Bitso['Retornos'] = np.log(Bitso['Average']/Bitso['Average'].shift(1))
    Bitso.dropna(inplace = True)
    Bitso['Estrategia'] = Bitso['Retornos']* Bitso['Posicion'].shift(1)
    Bitso.dropna(inplace = True)
    Bitso['Retacum'] = Bitso['Retornos'].cumsum().apply(np.exp)
    Bitso['Estracum'] = Bitso['Estrategia'].cumsum().apply(np.exp)

    #------------------Print it out
    fig = plt.figure(figsize=(14,14))

    ax1 = plt.subplot2grid((25,1),(0,0),rowspan = 8, colspan = 1)
    Bitso['Average'].plot(title = ('Estrategia PMs ' + str(pm1*10) + ', ' + str(pm2*10)))
    Bitso['PM1'].plot(ax=ax1)
    Bitso['PM2'].plot(ax=ax1)

    ax1.set_ylabel('Ultimo Precio, PM1, PM2')
    plt.xticks([],[])

    ax2 = plt.subplot2grid((25,1),(8,0), rowspan = 3, colspan = 1)
    Bitso['Posicion'].plot(ylim=[-1.1,1.1])
    ax2.set_ylabel('Posicion')
    plt.xticks([],[])

    ax3=plt.subplot2grid((25,1),(11,0),rowspan = 8, colspan = 1)
    Bitso[['Retacum', 'Estracum']].plot(ax = ax3)
    ax3.set_ylabel('Retornos Acumulados')
    plt.xticks([],[])
    
    return Bitso

In [ ]:
Best_Stra = 0
Best_PM1 = 0
Best_PM2 = 0
Last_PM1 = 43200 # 30 dias = 43200 min
Last_PM2 = 43200

the_path = os.getcwd()
data_path = os.path.join(the_path, '../data/processed-data/')
file_name = 'processed.csv'
file_path = data_path + file_name
df = pd.read_csv(file_path, index_col='Date')

df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.index = pd.to_datetime(df.index)
df_M = df.last('7M').copy()

#For cycle para ver cual es la mejor combinacion

for pm1 in np.arange(60, Last_PM1, 10):
    for pm2 in np.arange(1, Last_PM2, 10):
        if (pm1<pm2):
            Val_Stra = get_acum(pm1, pm2, df_M.copy())
            print ('\rPM1: {} | PM2: {} | Best: {}'.format(pm1,pm2,Best_Stra),end='')
            if (Val_Stra[0] > Best_Stra):
                Best_Stra = Val_Stra[0]
                Best_PM1 = pm1
                Best_PM2 = pm2
                print ('\nPM1: {} | PM2: {} | Best: {}'.format(pm1,pm2,Best_Stra),end='')
print('\nDONE!')

PM1: 60 | PM2: 61 | Best: 0
PM1: 60 | PM2: 351 | Best: 0.8599846263542594
PM1: 60 | PM2: 441 | Best: 0.8609015103187656
PM1: 60 | PM2: 451 | Best: 0.9412170090034656
PM1: 60 | PM2: 471 | Best: 1.0400026880266073
PM1: 60 | PM2: 491 | Best: 1.1930054385028155
PM1: 60 | PM2: 531 | Best: 1.3692231606368153
PM1: 60 | PM2: 541 | Best: 1.3786712706046513
PM1: 60 | PM2: 551 | Best: 1.422187097984463
PM1: 60 | PM2: 561 | Best: 1.4557769066484687
PM1: 60 | PM2: 1061 | Best: 1.5352787468558662
PM1: 60 | PM2: 1101 | Best: 1.594352212545536
PM1: 60 | PM2: 1111 | Best: 1.6811605484541907
PM1: 60 | PM2: 1121 | Best: 1.80290665092084
PM1: 60 | PM2: 1161 | Best: 1.8587933293307746
PM1: 60 | PM2: 1171 | Best: 1.9169363807477544
PM1: 60 | PM2: 1441 | Best: 2.141990715966317
PM1: 60 | PM2: 3051 | Best: 2.2221264082661447

In [ ]:
Best = best_strategy(Best_PM1,Best_PM2,df_M)
Best['Estracum'].tail(1) - Best['Retacum'].tail(1)